In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ConvLSTM2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
dataset_path = '/content/drive/MyDrive/CoMoFoD_small_v2'
au_dir = os.path.join(dataset_path, 'Au')
tp_dir = os.path.join(dataset_path, 'Tp')


In [ ]:
def load_images(directory, label):
    images = []
    labels = []


    for filename in tqdm(os.listdir(directory)):
        img_path = os.path.join(directory, filename)
        img = load_img(img_path, target_size=(128, 128))
        img = img_to_array(img) / 255.0
        images.append(img)
        labels.append(label)

    return images, labels


In [ ]:
authentic_images, authentic_labels = load_images(au_dir, 0)
tampered_images, tampered_labels = load_images(tp_dir, 1)

In [ ]:
X = np.concatenate((authentic_images, tampered_images), axis=0)
y = np.concatenate((authentic_labels, tampered_labels), axis=0)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape(-1, 1, 128, 128, 3)
X_val = X_val.reshape(-1, 1, 128, 128, 3)

In [ ]:
model3 = Sequential([
    # ConvLSTM layer
    ConvLSTM2D(filters=16, kernel_size=(3, 3), padding='same', return_sequences=False, input_shape=(1, 128, 128, 3)),

    # CNV layer with 32 filters and max pooling
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # CNV layer with 64 filters and max pooling
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # CNV layer with 128 filters and max pooling
    Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Global average pooling and fully connected layer
    GlobalAveragePooling2D(),
    Dense(units=1, activation='sigmoid')
])
model3.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()


In [ ]:
model3.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
loss, accuracy = model3.evaluate(X_val, y_val)
print(f'Validation loss: {loss:.4f}, accuracy: {accuracy:.4f}')

y_pred_prob = model3.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

In [ ]:
cm = confusion_matrix(y_val, y_pred)
print('Confusion Matrix:')
print(cm)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Authentic', 'Tampered'], yticklabels=['Authentic', 'Tampered'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


print('Classification Report:')
print(classification_report(y_val, y_pred, target_names=['Authentic', 'Tampered']))


In [ ]:
test_dataset_path = '/content/drive/MyDrive/MISD/MISD_test'
authentic_test_dir = os.path.join(test_dataset_path, 'Au')
tampered_test_dir = os.path.join(test_dataset_path, 'Tp')

def load_images(directory, label):
    images = []
    labels = []


    for filename in tqdm(os.listdir(directory)):
        img_path = os.path.join(directory, filename)
        img = load_img(img_path, target_size=(128, 128))
        img = img_to_array(img) / 255.0
        images.append(img)
        labels.append(label)

    return images, labels

authentic_test_images, authentic_test_labels = load_images(authentic_test_dir, 0)
tampered_test_images, tampered_test_labels = load_images(tampered_test_dir, 1)

X_test = np.concatenate((authentic_test_images, tampered_test_images), axis=0)
y_test = np.concatenate((authentic_test_labels, tampered_test_labels), axis=0)

X_test = X_test.reshape(-1, 1, 128, 128, 3)


loss, accuracy = model3.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}, accuracy: {accuracy:.4f}')


y_test_pred_prob = model3.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int).flatten()


cm_test = confusion_matrix(y_test, y_test_pred)
print('Test Confusion Matrix:')
print(cm_test)


plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['Authentic', 'Tampered'], yticklabels=['Authentic', 'Tampered'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Test Confusion Matrix')
plt.show()


print('Test Classification Report:')
print(classification_report(y_test, y_test_pred, target_names=['Authentic', 'Tampered']))